In [1]:
import os
from os.path import isfile, join, isdir
import json
from unidecode import unidecode

In [2]:
import json
import nltk
from corenlp import *
from nltk.tag import StanfordNERTagger
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import sentiwordnet as swn

In [3]:
from jsonrpc import ServerProxy, JsonRpc20, TransportTcpIp

class StanfordNLP:
    def __init__ (self):
        self.server = ServerProxy(JsonRpc20(),
                                 TransportTcpIp(addr=("127.0.0.1", 8080)))
    def parse (self, text):
        return json.loads(self.server.parse(text))
    
nlp = StanfordCoreNLP()

Loading Models: 5/5                                                            


In [4]:
dir_path = 'output'
#os.listdir(dir_path)
listdir = [dir_path + '/' + d for d in os.listdir(dir_path) if isdir(join(dir_path, d))]
listdir.sort()

print ('Cantidad de libros: ' + str(len(listdir)))

Cantidad de libros: 4


In [5]:
def new_obj():
    return {
        'verbos': [],
        'entidades': {
            'PERSON': set([]),
            'LOCATION': set([]),
            'ORGANIZATION': set([])
        }
    }

def my_unidecode(partial):
    new_partial = []
    for p in partial:
        new_p = []
        for s in p:
            obj = new_obj()
            for v in s['verbos']:
                obj['verbos'].append([unidecode(v[0]),v[1]])
            for o in s['entidades']['PERSON']:
                obj['entidades']['PERSON'].add(unidecode(o))
            for o in s['entidades']['LOCATION']:
                obj['entidades']['LOCATION'].add(unidecode(o))
            for o in s['entidades']['ORGANIZATION']:
                obj['entidades']['ORGANIZATION'].add(unidecode(o))
            new_p.append(obj)
        new_partial.append(new_p)
    return new_partial             

def read_book(book_path):
    list_files = [book_path + '/' + f for f in os.listdir(book_path) if isfile(join(book_path, f))]
    list_files.sort()
    book = []
    for path_file in list_files:
        with open(path_file) as f:
            partial = json.load(f)
            partial_decoded = my_unidecode(partial)
            book.append(partial_decoded)
    out = [p for book_partial in book for p in book_partial]
    return out

def read_test():
    list_files = ['test' + str(i) + '.json' for i in range(19)]
    book = []
    for path_file in list_files:
        with open(path_file) as f:
            partial = json.load(f)
            book.append(partial)
    out = [p for book_partial in book for p in book_partial]
    return out


In [6]:
def leer_book0():    
    with open('book0.json') as f:
        book0 = json.load(f)
        book0_text = [unidecode(p) for p in book0]
    book0_json = read_book(listdir[0])
    return book0_text, book0_json


In [7]:
book0 = leer_book0()
all_ent = {
    'PERSON': set(['Roland', 'Eddie', 'Jake', 'Oy',
              'Cuthbert', 'John', 'Randal', 'Aballah',
              'Jack', 'Blaine', 'Rhea', 'Andrew']),
    'LOCATION': set([]),
    'ORGANIZATION': set([])
}



In [8]:
def generar_entidades(book):
    for p in book:
        for s in p:
            for o in s['entidades']['PERSON']:
                all_ent['PERSON'].add(o)
            for o in s['entidades']['LOCATION']:
                all_ent['LOCATION'].add(o)
            for o in s['entidades']['ORGANIZATION']:
                all_ent['ORGANIZATION'].add(o)

In [9]:
generar_entidades(book0[1])

In [10]:
all_ent

{'LOCATION': {'Antichrist',
  'Charyou',
  'Commala',
  'Cort',
  'Cuthbert',
  'Damascus',
  'Earth',
  'End-World',
  'Garlan',
  'Israel',
  'JerichoHill',
  'Marten',
  'Mejis',
  'MohaineDesert',
  'Mount',
  'NewCanaan',
  'NewEarth',
  'NewJerseyTurnpike',
  'Nort',
  'NorthCentralPositronics',
  'Phoenician',
  'Pricetown',
  'Randal',
  'Roland',
  'SouthIslands',
  'Spain',
  'Taunton',
  'TauntonRoad',
  'Tull',
  'moon'},
 'ORGANIZATION': {'AMOCO',
  'ClayBlaisdellWestern',
  'Coca-Cola',
  'DarkTower',
  'EastWing',
  'Ford',
  'Gilead',
  'HorseheadNebula',
  'PiperSchool',
  "Sheb's",
  'TheNetwork',
  'Town',
  'West-Town'},
 'PERSON': {'Aballah',
  'Abednego',
  'Ahaz',
  'AileenRitter',
  'Alain',
  'AlainJohns',
  'Alan',
  'AlgulSiento',
  'Alice',
  'Allie',
  'AmyFeldon',
  'Andrew',
  'Arthur',
  'ArthurEld',
  'AuntMill',
  'Bathsheba',
  'Bert',
  'Blaine',
  'Bloomie',
  'Brown',
  'Castner',
  'Chambers',
  'Charles',
  'CharlesofCharles',
  'Charlie',
  'Cof

In [11]:
def concatenar(ori,cop):
    for so, sp in zip(ori,cop):
        for o in sp['entidades']['PERSON']:
            so['entidades']['PERSON'].add(o);

In [12]:
def reemplazar_referencias_parrafo(text, json_):
    #print 'Original text:'
    #print(text)
    #nlp = StanfordCoreNLP()
    result = json.loads(nlp.parse(text))
    tokenized_sentences = nltk.sent_tokenize(text)
    if (len(tokenized_sentences) == 1) :
        print " >  Solo hay una oracion, no existen relaciones."
        return
    
    tokenized_in_words=[nltk.word_tokenize(sentence) for sentence in tokenized_sentences]
    
    copia= [ new_obj() for s in json_]
    
    #print tokenized_in_words
    

    
    for block_to_replace in result["coref"]:
        #sentence_index=block_to_replace[0][1][1]
        #word_index=block_to_replace[0][1][2]
        #replace_sent=block_to_replace[0][1][0]
        
        word_to_replace=block_to_replace[0][0][0]

        if not word_to_replace in all_ent["PERSON"]:
            continue


        print ' -- Word_to_replace: ',word_to_replace

        for i,lines_to_replace in enumerate(block_to_replace):
            
            ix_sent=lines_to_replace[0][1]
            copia[ix_sent]['entidades']['PERSON'].add(word_to_replace)
            #sent_to_replace=lines_to_replace[0][0]
            #tokenized_sentences[ix_sent]=tokenized_sentences[ix_sent].replace(sent_to_replace,word_to_replace)

        #tokenized_sentences[sentence_index]=tokenized_sentences[sentence_index].replace(replace_sent,word_to_replace)
        print

    concatenar(json_,copia)
    return json_

In [13]:
for i,parT, parJ in zip(range(len(book0[0])),book0[0],book0[1]):

    print 'Parrafo ',i
    #print parT
    try:
        reemplazar_referencias_parrafo(parT,parJ)
    except:
        print
        print 'SKIPPING BECAUSE LIFE'
        print
    #break

Parrafo  0
Parrafo  1
 -- Word_to_replace:  Roland

 -- Word_to_replace:  Manni

Parrafo  2
Parrafo  3
Parrafo  4
Parrafo  5
Parrafo  6
Parrafo  7
Parrafo  8
Parrafo  9
Parrafo  10
Parrafo  11
Parrafo  12
Parrafo  13
Parrafo  14
Parrafo  15
Parrafo  16
Parrafo  17
Parrafo  18
 -- Word_to_replace:  Roland

 -- Word_to_replace:  Brown

 -- Word_to_replace:  Zoltan

Parrafo  19
 -- Word_to_replace:  Brown

 -- Word_to_replace:  Roland

Parrafo  20
Parrafo  21
Parrafo  22
 -- Word_to_replace:  Roland

Parrafo  23
Parrafo  24
 -- Word_to_replace:  Brown

Parrafo  25
 -- Word_to_replace:  Brown

Parrafo  26
Parrafo  27
Parrafo  28
Parrafo  29
Parrafo  30
Parrafo  31
Parrafo  32
Parrafo  33
 -- Word_to_replace:  Roland

Parrafo  34
 -- Word_to_replace:  Roland

Parrafo  35
Parrafo  36
Parrafo  37
Parrafo  38
Parrafo  39
 -- Word_to_replace:  Jude

Parrafo  40
Parrafo  41
 -- Word_to_replace:  Jake

Parrafo  42
Parrafo  43
Parrafo  44
Parrafo  45
Parrafo  46
 -- Word_to_replace:  Roland

Parra

ERROR:corenlp:Error: Timeout with input '"O Jesus Savior--" "It was him who took our Lord up on the mountain--" "Yes--" "It was him that tempted him and shewed him all the world and the world's pleasures--" "Yesss--" "It's him that will return when Last Times come on the world . . . and they are coming, my brothers and sisters, can't you feel they are?"
'



SKIPPING BECAUSE LIFE

Parrafo  133
Parrafo  134
Parrafo  135
Parrafo  136

SKIPPING BECAUSE LIFE

Parrafo  137
Parrafo  138
Parrafo  139
Parrafo  140
 -- Word_to_replace:  Jonson

Parrafo  141
Parrafo  142
Parrafo  143
Parrafo  144
Parrafo  145
Parrafo  146
Parrafo  147
Parrafo  148
Parrafo  149
Parrafo  150
Parrafo  151
Parrafo  152
Parrafo  153
Parrafo  154
Parrafo  155
Parrafo  156
Parrafo  157
Parrafo  158
Parrafo  159
Parrafo  160
Parrafo  161
 -- Word_to_replace:  Kennerly

 -- Word_to_replace:  Roland

Parrafo  162
 -- Word_to_replace:  Roland

 -- Word_to_replace:  Soobie

 -- Word_to_replace:  Allie

Parrafo  163
Parrafo  164
Parrafo  165
 -- Word_to_replace:  Allie

Parrafo  166
Parrafo  167
Parrafo  168
Parrafo  169
Parrafo  170
 -- Word_to_replace:  Sheb

Parrafo  171
Parrafo  172
Parrafo  173
Parrafo  174
Parrafo  175
Parrafo  176
Parrafo  177
Parrafo  178
Parrafo  179
Parrafo  180
Parrafo  181
Parrafo  182
Parrafo  183
 -- Word_to_replace:  Brown

Parrafo  184
 -- Word_

Parrafo  450
 -- Word_to_replace:  David

Parrafo  451
Parrafo  452
 -- Word_to_replace:  Cort

Parrafo  453
 -- Word_to_replace:  Cort

 -- Word_to_replace:  David

 -- Word_to_replace:  Jakes

Parrafo  454
Parrafo  455
 -- Word_to_replace:  Cort

Parrafo  456
 -- Word_to_replace:  Cort

Parrafo  457
 -- Word_to_replace:  Jake

Parrafo  458
Parrafo  459
 -- Word_to_replace:  Cort

Parrafo  460
 -- Word_to_replace:  Cort

Parrafo  461
Parrafo  462
 -- Word_to_replace:  Jake

Parrafo  463
 -- Word_to_replace:  Jake

 -- Word_to_replace:  Cort

Parrafo  464
 -- Word_to_replace:  Jake

Parrafo  465
Parrafo  466
Parrafo  467
Parrafo  468
 -- Word_to_replace:  Roland

Parrafo  469
Parrafo  470
Parrafo  471
Parrafo  472
 -- Word_to_replace:  Roland

 -- Word_to_replace:  Cuthbert

 -- Word_to_replace:  Alain

Parrafo  473
 -- Word_to_replace:  Roland

Parrafo  474
 -- Word_to_replace:  Jake

Parrafo  475
Parrafo  476
Parrafo  477
 -- Word_to_replace:  Roland

Parrafo  478
 -- Word_to_replace

In [40]:
for p in book0[1]:
    for o in p:
        o['entidades']['PERSON'] = list(o['entidades']['PERSON'])
        o['entidades']['LOCATION'] = list(o['entidades']['LOCATION'])
        o['entidades']['ORGANIZATION'] = list(o['entidades']['ORGANIZATION'])
        



In [41]:

with open('book0_result.json','w')as out:
    json.dump(book0[1],out)